In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
training_data=datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data=datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [5]:
test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [6]:
batch_size=64
train_dataloader=DataLoader(training_data, batch_size=batch_size)
test_dataloader=DataLoader(test_data, batch_size=batch_size)

In [8]:
for X, y in test_dataloader:
    print(f"Shape of X [N,C,H,W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N,C,H,W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [9]:
device=(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using mps device


**Define Model**

In [42]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
            #nn.Sigmoid()
        )
        
    def forward(self, x):
        x=self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
        

In [43]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): Sigmoid()
  )
)


**Optimizing Model Parameters**

In [44]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=1e-3)

In [45]:
def train(dataloader, model, loss_fn, optimizer):
    size=len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        #Compute prediction error
        pred=model(X)
        loss=loss_fn(pred,y)
        
        #Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100==0:
            loss, current=loss.item(), (batch+1)*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [46]:
def test(dataloader, model, loss_fn):
    size=len(dataloader.dataset)
    num_batches=len(dataloader)
    model.eval()
    test_loss, correct= 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y=X.to(device), y.to(device)
            pred=model(X)
            test_loss+=loss_fn(pred,y).item() #.item() converts tensor to number
            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [47]:
epochs=5
for t in range(epochs):
    print(f"Epoch {t+1} \n ---------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Finito!")

Epoch 1 
 ---------------
loss: 2.305916 [   64/60000]
loss: 2.304993 [ 6464/60000]
loss: 2.302169 [12864/60000]
loss: 2.301659 [19264/60000]
loss: 2.300867 [25664/60000]
loss: 2.301015 [32064/60000]
loss: 2.300403 [38464/60000]
loss: 2.298708 [44864/60000]
loss: 2.300131 [51264/60000]
loss: 2.296578 [57664/60000]
Test Error: 
 Accuracy: 17.8%, Avg loss: 2.296288 

Epoch 2 
 ---------------
loss: 2.297253 [   64/60000]
loss: 2.297481 [ 6464/60000]
loss: 2.292549 [12864/60000]
loss: 2.294377 [19264/60000]
loss: 2.292873 [25664/60000]
loss: 2.290359 [32064/60000]
loss: 2.292867 [38464/60000]
loss: 2.288955 [44864/60000]
loss: 2.291458 [51264/60000]
loss: 2.287573 [57664/60000]
Test Error: 
 Accuracy: 29.3%, Avg loss: 2.287102 

Epoch 3 
 ---------------
loss: 2.288278 [   64/60000]
loss: 2.289523 [ 6464/60000]
loss: 2.282396 [12864/60000]
loss: 2.286393 [19264/60000]
loss: 2.283882 [25664/60000]
loss: 2.278696 [32064/60000]
loss: 2.284475 [38464/60000]
loss: 2.277981 [44864/60000]
loss: 

In [ ]:
#torch.save(model.state_dict(),"model.pth")
#print("Saved PyTorch Model State to model.pth")

In [ ]:
#model=NeuralNetwork().to(device)
#model.load_state_dict(torch.load("model.pth"))

**Make Predictions**

In [49]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x,y=test_data[2][0], test_data[2][1]
with torch.no_grad():
    x=x.to(device)
    pred=model(x)
    predicted, actual=classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Trouser", Actual: "Trouser"
